# PostScriptML: Cleaning Notebook
## by Dolci Key 

## Import Libraries

In [1]:
import pandas as pd
import numpy as py

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import shap

import glob
import os

from keras import models, layers
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import inception_v3, mobilenet_v2, vgg19, densenet
from keras.layers import Dense,GlobalAveragePooling2D
import tensorflow as tf
from PIL import Image, ImageOps, ImageFilter

import os, shutil
import argparse
import sys

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics.pairwise import cosine_similarity

import pickle

# Data 

### For this project, these are my own photos taken with a Nikon Z7. Photos have been pre-labeled as select and reject. I will be using an S3 Bucket to store the data as these images are .NEF files, large uncompressed images, which take up between 40-60 MB each. I used these images because professional photographers always shoot in RAW and these will be what they are looking through for post production photograph selections. 

### First I will read in the images and make sure the images are not corrupted.


In [ ]:
select = 'PostScript/DATA/select'
reject = 'PostScript/DATA/reject'
